In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn

from centerloss import *

def main():
  classCount = 14
  model = LightningChestXrayCnnClassifierInceptionV3(classCount)
  trainer = pl.Trainer(max_epochs=15)
  trainer.fit(model)

class LitModule(pl.LightningModule):
  
  def __init__(self, classCount, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.encoder = BERT_encoder
    self.classCount = classCount
    # ... more initialization elided for simplicity

    self.loss_center = num_classes=?, feat_dim=?, use_gpu=True)
    self.loss1 = torch.nn.CrossEntropyLoss()
    self.loss2 = torch.nn.CrossEntropyLoss()
    self.loss3 = torch.nn.CrossEntropyLoss()
    self.loss4 = torch.nn.CrossEntropyLoss()
    self.loss5 = torch.nn.CrossEntropyLoss()
    self.loss6 = torch.nn.CrossEntropyLoss()
    self.loss7 = torch.nn.CrossEntropyLoss()

  def configure_optimizers(self):
      optimizer_centerloss = torch.optim.SGD(center_loss.parameters(), lr=0.5) #alpha = 1
      optimizer_crossentropy = torch.optim.Adam()


    # ... elided for simplicity

  def train_dataloader(self):
    # ... elided for simplicity

  def training_step(self, batch, batch_idx):
    x, y = batch
    input = self.trans_train(x)

    targets = [i for i in y]

    for t in targets:
        classifier_out = 

    input = input.to(dtype=torch.float32, device=self.device)
    varInput = torch.autograd.Variable(input)
    varTarget1 = torch.autograd.Variable(input)
    varTarget2 = torch.autograd.Variable(y)

    varOutput1, varOutput2 = self.module.forward(varInput)
    classifierOut1, classifierOut2 = varOutput2

    # Transformer model center loss:
    lossvalue1 = self.loss1(varOutput1, varTarget1)
    # weighting between main and aux branch of inception model:
    lossvalue2 = 0.8 * self.loss2(classifierOut1, varTarget2) + 0.2 * self.loss2(classifierOut2, varTarget2)  

    losses = [loss_center,loss_val1,loss_val2,loss_val3,loss_val4,loss_val5,loss_val6,loss_val7]
    # weighting btween MSE and BCE respectively:
    loss = sum(losses)

    output = {
      'loss': loss,  # required
    }
    return output

def trans_train(self, x):
  # ...  some image transformations elided for simplicity



if __name__ == "__main__":
    main()



